In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets,transforms
from PIL import Image
from tqdm import tqdm
import os
import random
import cv2

/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


切訓練 測試 

train/fit 2400 train/nofit 2375

test/fit 600 test/nofit 610

In [26]:
os.listdir("../p")

['.DS_Store', 'a', 'c', 'd', 'b']

In [24]:
transform=transforms.Compose([
   transforms.CenterCrop([560,450]),
    transforms.Resize([224,224]),
])

os.mkdir("train")
os.mkdir("test")
os.mkdir("train/fit")
os.mkdir("test/fit")
os.mkdir("train/nofit")
os.mkdir("test/nofit")

train_fit_sets = random.sample(os.listdir("OB outfit sets/fit"), 3000)
test_fit_sets = random.sample(train_fit_sets, 600)
train_nofit_sets = random.sample(os.listdir("OB outfit sets/nofit"), 2985)
test_nofit_sets = random.sample(train_nofit_sets, 610)

for x in test_fit_sets:
    train_fit_sets.remove(x)
for x in test_nofit_sets:
    train_nofit_sets.remove(x)
    
ct = 0
for fit_set in train_fit_sets:
    for f_item in os.listdir(f'OB outfit sets/fit/{fit_set}'):
        path = "OB outfit sets/fit/"+fit_set+"/"+f_item
        pil_img=Image.open(path)
        t=transform(pil_img)
        os.mkdir("train/fit/fit"+ct)
        save_dir="train/fit/fit"+ct+"/"+f_item
        ct += 1
        t.save(save_dir)
ct = 0
for fit_set in test_fit_sets:
    for f_item in os.listdir(f'OB outfit sets/fit/{fit_set}'):
        path = "OB outfit sets/fit/"+fit_set+"/"+f_item
        pil_img=Image.open(path)
        t=transform(pil_img)
        os.mkdir("test/fit/fit"+ct)
        save_dir="test/fit/fit"+ct+"/"+f_item
        ct += 1
        t.save(save_dir)
ct = 0
for nofit_set in train_nofit_sets:
    for nof_item in os.listdir(f'OB outfit sets/nofit/{nofit_set}'):
        path = "OB outfit sets/fit/"+nofit_set+"/"+nof_item
        pil_img=Image.open(path)
        t=transform(pil_img)
        os.mkdir("train/nofit/nofit"+ct)
        save_dir="train/nofit/nofit"+ct+"/"+nof_item
        ct += 1
        t.save(save_dir)
ct = 0
for nofit_set in test_nofit_sets:
    for nof_item in os.listdir(f'OB outfit sets/nofit/{nofit_set}'):
        path = "OB outfit sets/fit/"+nofit_set+"/"+nof_item
        pil_img=Image.open(path)
        t=transform(pil_img)
        os.mkdir("test/nofit/nofit"+ct)
        save_dir="test/nofit/nofit"+ct+"/"+nof_item
        ct += 1
        t.save(save_dir)

[1, 6, 8]

---

### loss function參考

In [2]:
#loss function參考1
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Load VGG16 model as the backbone
        self.backbone = models.vgg16(pretrained=True)
        # Remove the last layer (the classification layer) of VGG16
        self.backbone.classifier = nn.Sequential(*list(self.backbone.classifier.children())[:-1])
        # Add a fully connected layer with 512 output units for Siamese network
        self.fc = nn.Linear(512, 1)

    def forward_once(self, x):
        # Pass input image through the VGG16 backbone
        output = self.backbone(x)
        # Flatten the output
        output = output.view(output.size()[0], -1)
        # Pass through the fully connected layer
        output = self.fc(output)
        return output

    def forward(self, x1, x2):
        # Pass the first image through the network
        output1 = self.forward_once(x1)
        # Pass the second image through the network
        output2 = self.forward_once(x2)
        # Return the absolute difference between the two outputs
        distance = torch.abs(output1 - output2)
        return distance

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        # Compute the contrastive loss
        loss = torch.mean((1 - label) * torch.pow(distance, 2) +
                          (label) * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2))
        return loss

In [ ]:
#loss function參考2
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, target):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - target) * torch.pow(euclidean_distance, 2) +
                                      target * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

In [ ]:
#loss function參考3
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss
    Takes embeddings of two samples and a target label == 1 if samples are from the same class and label == 0 otherwise
    """

    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, output1, output2, target, size_average=True):
        distances = (output2 - output1).pow(2).sum(1)  # squared distances
        losses = 0.5 * (target.float() * distances +
                        (1 + -1 * target).float() * F.relu(self.margin - (distances + self.eps).sqrt()).pow(2))
        return losses.mean() if size_average else losses.sum()

---

###

---

In [41]:
#define dataset
#data_dir應該要擺成train/fit/data_pair/2張圖片
transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])
batch_size = 64
train_dir = "檔案位置"           
#定義數據集
train_datasets = datasets.ImageFolder(train_dir, transform=transform)
#加載數據集
train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=False)

# val_dir = "test"		
# val_datasets = datasets.ImageFolder(val_dir, transform=transform2)
# val_dataloader = torch.utils.data.DataLoader(val_datasets, batch_size=batch_size, shuffle=False)

In [18]:
#define data loader
train_root_dir = 'train'
train_pairs_list = []

for fit_ in ['fit', 'nofit']:
    fit_dir = f"{train_root_dir}/{fit_}"
    for pair in os.listdir(fit_dir)[1:]:
        imgs = os.listdir(f"{fit_dir}/{pair}")[1:]
        train_pairs_list.append([
                            f"{fit_dir}/{pair}/{imgs[0]}", 
                            f"{fit_dir}/{pair}/{imgs[1]}",
                            int(fit_ == 'fit')
                            ])
        # train_pairs_list_label.append(int(fit_ != 'fit'))
        # for img in imgs:
        #     train_pairs_list.append(f"{fit_dir}/{img}")
        #     train_pairs_list_label.append(int(fit_ != 'fit'))
#print(*train_pairs_list, sep = '\n')
random.shuffle(train_pairs_list)
train_pairs_list
# for i, j in zip(train_pairs_list, train_pairs_list_label):
#     print(i, j)
# print(*train_pairs_list_label, sep = '\n')

[['train/fit/pair2/fit377.png', 'train/fit/pair2/fit376.png', 0],
 ['train/fit/pair1/fit374.png', 'train/fit/pair1/fit375.png', 0],
 ['train/nofit/pair1/fit381.png', 'train/nofit/pair1/fit382.png', 1],
 ['train/nofit/pair2/fit384.png', 'train/nofit/pair2/fit383.png', 1]]

In [53]:
def train_pairs_loader(train_data, batch_size):
    cnt = len(train_data) // batch_size + int(len(train_data) % batch_size != 0)
    for i in range(cnt):
        temp = i * (batch_size)
        yield train_data[temp : temp + batch_size]

In [54]:
# x = my_imgs_pair_loader(list(range(10)), 6)
x = train_pairs_loader(train_pairs_list, 2)
for i in x:
    print(i)

[['train/fit/pair1/fit374.png', 'train/fit/pair1/fit375.png', 0], ['train/fit/pair2/fit377.png', 'train/fit/pair2/fit376.png', 0]]
[['train/nofit/pair1/fit381.png', 'train/nofit/pair1/fit382.png', 1], ['train/nofit/pair2/fit384.png', 'train/nofit/pair2/fit383.png', 1]]


---

### 胡亂測試

In [58]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])

img = Image.open("train/nofit/pair2/fit384.png")
img_t = transform(img)
img_t

[ WARN:0@823.664] global /Users/runner/miniforge3/conda-bld/libopencv_1661642967242/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('train/nofit/fit384.png'): can't open/read file: check file path/integrity


TypeError: pic should be Tensor or ndarray. Got <class 'NoneType'>.

In [4]:
transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])
train_dataloader = torch.utils.data.DataLoader(pairs_list,batch_size=1)
for i in train_dataloader:
    print(i)

[('train/fit/pair1/fit374.png',), ('train/fit/pair1/fit375.png',), tensor([0])]
[('train/fit/pair2/fit377.png',), ('train/fit/pair2/fit376.png',), tensor([0])]
[('train/nofit/pair1/fit381.png',), ('train/nofit/pair1/fit382.png',), tensor([1])]
[('train/nofit/pair2/fit384.png',), ('train/nofit/pair2/fit383.png',), tensor([1])]


In [42]:
#測試
print(train_datasets[5755])
print(train_datasets[5755][0].shape)
print(f'label是{train_datasets[5755][1]}')


(tensor([[[0.8118, 0.8118, 0.8118,  ..., 0.8118, 0.8118, 0.8039],
         [0.8196, 0.8196, 0.8196,  ..., 0.8118, 0.8118, 0.8039],
         [0.8196, 0.8275, 0.8196,  ..., 0.8118, 0.8118, 0.8039],
         ...,
         [0.7961, 0.7961, 0.7961,  ..., 0.7490, 0.7098, 0.6471],
         [0.7961, 0.7961, 0.7961,  ..., 0.7412, 0.7020, 0.6314],
         [0.7961, 0.7882, 0.7961,  ..., 0.7412, 0.6941, 0.6157]],

        [[0.8039, 0.8039, 0.8039,  ..., 0.7882, 0.7882, 0.7882],
         [0.8118, 0.8118, 0.8118,  ..., 0.7882, 0.7882, 0.7882],
         [0.8118, 0.8118, 0.8118,  ..., 0.7882, 0.7882, 0.7882],
         ...,
         [0.7098, 0.7098, 0.7098,  ..., 0.6392, 0.6000, 0.5137],
         [0.7098, 0.7098, 0.7098,  ..., 0.6392, 0.5843, 0.4902],
         [0.7098, 0.7020, 0.7098,  ..., 0.6314, 0.5765, 0.4745]],

        [[0.7882, 0.7882, 0.7882,  ..., 0.7490, 0.7490, 0.7490],
         [0.7961, 0.7961, 0.7961,  ..., 0.7490, 0.7490, 0.7412],
         [0.8039, 0.7961, 0.7961,  ..., 0.7490, 0.7490, 0

In [44]:
#測試
train_dir = "../compatibility task/filtered data"           
train_datasets = datasets.ImageFolder(train_dir)
img ,label= train_datasets[5755]
print(label)
print(img.format, img.size, img.mode)
img.show()


6
None (570, 760) RGB


In [59]:
#測試
train_dir = "train"           
train_datasets = datasets.ImageFolder(train_dir)
img ,label= train_datasets[4]
print(label)
img.show()

1


In [21]:
#測試
transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])
train_dir = "train"   
batch_size = 1
train_datasets = datasets.ImageFolder(train_dir,transform=transform)
train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=False)

In [25]:
#測試
for i in train_dataloader:
    print(i[0],i[1])



tensor([[[[ 0.8510,  0.8510,  0.8510,  ...,  0.4118,  0.2941,  0.3490],
          [ 0.8510,  0.8510,  0.8510,  ...,  0.3961,  0.2941,  0.3647],
          [ 0.8588,  0.8510,  0.8510,  ...,  0.3882,  0.3020,  0.3804],
          ...,
          [ 0.8039,  0.8118,  0.8118,  ...,  0.5137,  0.5216,  0.5137],
          [ 0.8039,  0.8039,  0.8039,  ...,  0.5137,  0.5137,  0.5059],
          [ 0.8039,  0.8039,  0.8118,  ...,  0.5059,  0.4980,  0.4980]],

         [[ 0.7882,  0.7882,  0.7882,  ...,  0.0196, -0.3412, -0.3412],
          [ 0.7882,  0.7882,  0.7882,  ...,  0.0039, -0.3333, -0.3255],
          [ 0.7961,  0.7882,  0.7882,  ...,  0.0039, -0.3255, -0.3098],
          ...,
          [ 0.6863,  0.6784,  0.6784,  ...,  0.4039,  0.4118,  0.4196],
          [ 0.6863,  0.6784,  0.6784,  ...,  0.4039,  0.4118,  0.4118],
          [ 0.6784,  0.6784,  0.6784,  ...,  0.4039,  0.3961,  0.4039]],

         [[ 0.7647,  0.7647,  0.7647,  ..., -0.2157, -0.6157, -0.6471],
          [ 0.7647,  0.7647,  

---

### 定義模型

In [11]:
#define embeddingnet which is used to extract image emabedding 
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.features = models.vgg16(pretrained=True)
        # ------------------------------------------------------------------------------
        # Remove the last layer (the classification layer) of VGG16 第一種寫法 把原本VGG的最後一層fc layer刪掉 直接線性輸出
        # self.backbone.classifier = nn.Sequential(*list(self.backbone.classifier.children())[:-1])
        # self.features = self.backbone.classifier
        # self.fc = nn.Linear(512, 1)
        # ------------------------------------------------------------------------------
        # 第二種寫法 
        self.features.classifier = nn.Sequential()
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, 10),#原num_classes 這裡要用多少維輸出來代表這個pic
        )
    def forward(self, x):
        #經過VGG提取特徵
        output = self.backbone(x)
        #flatten
        output = output.view(output.size(0)*output.size(1))
        #經過fc層
        output = self.fc(x)
        return output
        
    # def get_embedding(self, x):
        # return self.forward(x)


In [12]:
print(EmbeddingNet())    

/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/anaconda3/envs/torch8019/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EmbeddingNet(
  (features): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [18]:
#define siamesenet which have 2 inputs
class SiameseNet(nn.Module):
    def __init__(self, EmbeddingNet):
        super(SiameseNet, self).__init__()
        self.EmbeddingNet = EmbeddingNet
        
    def forward(self, x1, x2):
        output1 = self.EmbeddingNet(x1)
        output2 = self.EmbeddingNet(x2)
        return output1, output2

    # def Siamese(self, x1, x2):
    #     return self.SiameseNet(x1, x2)

In [ ]:
embedding_net = EmbeddingNet()
siamese_net = SiameseNet(embedding_net)
print(siamese_net)

In [31]:
#contrastive loss formula y(d**2) + (1-y)*max(margin-d,0)**2
#distance 0.5 similar label=1
print(1*(0.5**2) + (1-1)*max((2-0.5),0)**2) #0.25
#distance 1 dissimilar
print(0*(1**2) + (1-0)*max((2-1),0)**2) #1

#因為0是搭的 1是不搭的所以改成這樣
print(1*(1**2) + (1*1)*max((2-1),0)**2) #2

print(0*(0.5**2) + (1*0)*max((2-0.5),0)**2) #0
def closs(d,y,margin):
    loss = 1*(d**2) + (1-y)*max((margin-d),0)**2
    return loss

0.25
1
2
0.0


In [19]:
print(1*(0.5**2) + (1-1)*max((2-0.5),0)**2) #2

print(0*(3**2) + (1-0)*max((3-2),0)**2) #0

0.25
1


In [13]:
#define contrastive loss
#loss function reference https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2): #margin要自己定義 2
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        #self.eps = 1e-9
    
    #這邊可以改成不同距離計算方式 這裡先用歐幾里德距離計算
    def forward(self, embedding1, embedding2, label):
        distance = F.pairwise_distance(embedding1, embedding2, p=2)
        #本來最前面是(1-label)
        loss = torch.mean(label * torch.pow(distance, 2) + (1-label) * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2))
        return loss

IndentationError: expected an indented block (3052328748.py, line 3)

In [ ]:
#training 
learning_rate = 1e-5
epoch = 50
batch_size = 32

bar_len = []
for batch in train_pairs_loader(train_pairs_list,batch_size):
    bar_len.append(batch)

if torch.cuda.is_available():
    siamese_net.cuda()
params = [{'params': md.parameters()} for md in siamese_net.children()
          if md in [siamese_net.EmbeddingNet.fc]]
optimizer = optim.Adam(siamese_net.parameters(), lr=learning_rate)
loss_func = ContrastiveLoss()

train_dataloader = train_pairs_loader(train_pairs_list, batch_size)
siamese_net.train()

transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])

# threshold = 0.5
for i in range(epoch):
    print(f'epoch {i+1}')
    contrastive_loss = 0.
    pbar=tqdm(total=len(bar_len))
    train_loss = 0.
    train_d_list = []
    train_label_list = []
    # train_minmax_list = []
    # train_ans_list = []
    for batch_ in train_pairs_loader(train_pairs_list, batch_size):
        for img1_p, img2_p, label_ in batch_ :
            img1 = Image.open(img1_p)
            img2 = Image.open(img2_p)
            img1_t = transform(img1)
            img2_t = transform(img2)
            x1 = Variable(img1_t).cuda()
            x2 = Variable(img2_t).cuda()
            label_ = torch.tensor(label_,dtype=torch.int8)
            label = Variable(label_).cuda()
            optimizer.zero_grad()
            embedding1, embedding2 = siamese_net(x1, x2)
            loss = loss_func(embedding1, embedding2, label)
            train_loss += loss.item()
            # train_d_list.append(loss.item())
            # train_label_list.append(label)
            # contrastive_loss += loss.item()
            loss.backward()
            optimizer.step()
            pbar.update(1)
        pbar.close()

    # big = max(train_d_list)
    # small = min(train_d_list)
    # correct=0
    # for idx,j in enumerate(train_d_list):
    #     if j == big:
    #         j_nor = 1
    #         # train_minmax_list.append(j_nor)
    #     elif j == small:
    #         j_nor = 0
    #         # train_minmax_list.append(j_nor)
    #     j_nor = (j-small)/(big-small)
    #     # train_minmax_list.append(j_nor)
    #     if j_nor < threshold:
    #         ans = 0
    #     else:
    #         ans = 1
        
    #     if ans == train_label_list[idx]:
    #         correct += 1
    # # for idx,ans in enumerate(train_ans_list):
    # #     if ans == train_label_list[idx]:
    # #         correct += 1
    # print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(train_datasets)), correct / (len(train_datasets))))
    print('Train Loss: {:.6f}'.format(train_loss / (len(train_pairs_list))))


In [ ]:
torch.save(EmbeddingNet.state_dict(), "best.pt")
torch.savel(EmbeddingNet, "best.pt")